In [1]:
print('brian')

brian


In [2]:
 !pip install -q transformers datasets peft accelerate bitsandbytes trl

In [3]:
#loading the dataset
from google.colab import files
uploaded = files.upload()

Saving kenya_mmf_data.json to kenya_mmf_data.json


In [4]:
import json

with open("kenya_mmf_data.json", "r") as f:
    mmf_data = json.load(f)

print(f"Loaded {len(mmf_data)} examples.")
print(mmf_data[0])  # show one example

Loaded 20 examples.
{'instruction': 'What is a money market fund in Kenya?', 'input': '', 'output': 'A money market fund (MMF) in Kenya is a type of collective investment scheme that invests in low-risk, short-term debt instruments such as treasury bills, commercial paper, and fixed deposits. MMFs are designed to offer capital preservation and stable returns, and are regulated by the Capital Markets Authority (CMA).'}


In [5]:
#dataset formatting
from datasets import Dataset

#format each example into a single string
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    }

# Convert list of dicts to Hugging Face Dataset
dataset = Dataset.from_list(mmf_data)

formatted_dataset = dataset.map(format_example)

#sampling
formatted_dataset[0]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

{'instruction': 'What is a money market fund in Kenya?',
 'input': '',
 'output': 'A money market fund (MMF) in Kenya is a type of collective investment scheme that invests in low-risk, short-term debt instruments such as treasury bills, commercial paper, and fixed deposits. MMFs are designed to offer capital preservation and stable returns, and are regulated by the Capital Markets Authority (CMA).',
 'text': '### Instruction:\nWhat is a money market fund in Kenya?\n\n### Input:\n\n\n### Response:\nA money market fund (MMF) in Kenya is a type of collective investment scheme that invests in low-risk, short-term debt instruments such as treasury bills, commercial paper, and fixed deposits. MMFs are designed to offer capital preservation and stable returns, and are regulated by the Capital Markets Authority (CMA).'}

In [6]:
#loading the tokenizer
from transformers import AutoTokenizer
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-68691a90-13cd7b7a688748e748f84ae5;57521d38-6d6e-491d-aba5-db9de8b243f1)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
#tokenize the formatted dataset
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
tokenized_dataset = formatted_dataset.map(tokenize, batched=True)
tokenized_dataset[0]